This Python notebook is used for calculating the the Positron Emission Tomography (PET) and the Computed Tomography (CT) statistics such as Hounsfield Unit (HU) mean, Hounsfield Unit Standard Deviation, Standardized Uptake Value (SUV) mean, SUV peak, and SUV max. It also computes the volume of the organs of interest in ml.
1. Import the necessary libraries
2. Input files that are required for statistics computation. 
((i) DICOM file PET
(i) DICOM file CT
(iii) NIfTI file PET
(iv) NIfTI file CT
(v) Segmented organs)
3. Extract the requird metadata from the DICOM and store it. (Explanation: We are performing all of our calculations on NIfTI file and since NIfTI file does not have our necessary metadata in them, we need to extract the required metadata from the DICOM and use it for our calculations with NIfTI file)
4. Main functions
5. Output storage path (Excel file path to store the output)
6. Main code

1. Import the necessary libraries

In [1]:
import pydicom
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import datetime
from datetime import timedelta
from scipy.ndimage import median_filter

2. Required input files path:

In [ ]:
#DICOM file paths
pet_dicom_file_path = "C:/Personal/ABX/patient_2/DICOM/PETCT_4ef69de4e1/10-25-2002-NA-PET-CT Teilkoerper  primaer mit KM-18049/9.000000-PET corr.-01846/1-001.dcm"
ct_dicom_file_path = "C:/Personal/ABX/patient_2/DICOM/PETCT_4ef69de4e1/10-25-2002-NA-PET-CT Teilkoerper  primaer mit KM-18049/4.000000-THA p.v.3-96290/1-001.dcm"

# NIfTI file paths
nifti_file_path_pet = "C:/Personal/ABX/patient_2/NIFTI/PETCT_4ef69de4e1/10-25-2002-NA-PET-CT Teilkoerper  primaer mit KM-18049/PET.nii.gz"
nifti_file_path_ct = "C:/Personal/ABX/patient_2/NIFTI/PETCT_4ef69de4e1/10-25-2002-NA-PET-CT Teilkoerper  primaer mit KM-18049/CTres.nii.gz"

# Segmented organs pathh
segmented_organs_folder = "C:/Personal/ABX/patient_2/segmentations_pre"

#Get a list of all NIfTI files in the "segmented_organs" folder
segmented_nifti_paths = [os.path.join(segmented_organs_folder, file) for file in os.listdir(segmented_organs_folder) if file.endswith(".nii.gz")]

3. Extract the required metadata from the DICOM and store it in an array named "metadata"

In [2]:
def get_data_element_value(data_element):
    # If the element is None, return 'Attribute not found'
    if data_element is None:
        return 'Attribute not found'
    # If the element is a pydicom DataElement, return its value
    elif isinstance(data_element, pydicom.dataelem.DataElement):
        return data_element.value
    # If the element is already a value (e.g., a string), return it directly
    else:
        return data_element
    
def dicom_time_to_timedelta(dicom_time_str):
    #Time is in HHMMSS.FFFFFF format
    if dicom_time_str:
        hours = int(dicom_time_str[0:2])
        minutes = int(dicom_time_str[2:4])
        seconds = float(dicom_time_str[4:])
        return timedelta(hours=hours, minutes=minutes, seconds=seconds)
    return None

def extract_metadata(pet_dicom_file_path, ct_dicom_file_path):
    pet_dicom_data = pydicom.dcmread(pet_dicom_file_path)
    ct_dicom_data = pydicom.dcmread(ct_dicom_file_path)

    metadata = {
        'patient_id': get_data_element_value(pet_dicom_data.PatientID),
        'weight': get_data_element_value(pet_dicom_data.PatientWeight),
        'series_date': get_data_element_value(pet_dicom_data.SeriesDate),
        'series_time': get_data_element_value(pet_dicom_data.SeriesTime),
        'acquisition_date': get_data_element_value(pet_dicom_data.AcquisitionDate),
        'acquisition_time': get_data_element_value(pet_dicom_data.AcquisitionTime),
        'rescale_slope_pet': get_data_element_value(pet_dicom_data.get((0x0028, 0x1053))),
        'rescale_intercept_pet': get_data_element_value(pet_dicom_data.get((0x0028, 0x1052))),
        'rescale_slope_ct': get_data_element_value(ct_dicom_data.get((0x0019, 0x1092))),
        'rescale_intercept_ct': get_data_element_value(ct_dicom_data.get((0x0019, 0x1093)))
    }
    metadata['series_time'] = dicom_time_to_timedelta(metadata['series_time'])
    metadata['acquisition_time'] = dicom_time_to_timedelta(metadata['acquisition_time'])
    # Extract Radionuclide Information from the sequence
    rad_info_sequence = pet_dicom_data.get((0x0054, 0x0016), None)
    if rad_info_sequence:
        rad_info_item = rad_info_sequence[0]
        metadata['radionuclide_total_dose'] = get_data_element_value(rad_info_item.get((0x0018, 0x1074)))
        metadata['radionuclide_half_life'] = get_data_element_value(rad_info_item.get((0x0018, 0x1075)))

    return metadata

# Read DICOM metadata
metadata = extract_metadata(pet_dicom_file_path, ct_dicom_file_path)

4. Main functions to calculate the statistics:

In [3]:
def calculate_hu_metrics(ct_nifti_path, segmented_nifti_path, rescale_slope_ct, rescale_intercept_ct):
    # Load CT NIfTI file
    ct_img = nib.load(ct_nifti_path)
    ct_data = ct_img.get_fdata()

    # Load segmented NIfTI file for HU calculation and mask metrics
    segmented_img = nib.load(segmented_nifti_path)
    segmented_data = segmented_img.get_fdata()

    # Calculate HU values for the organ region
    hu_values_organ = (ct_data * segmented_data * rescale_slope_ct) + rescale_intercept_ct

    # Initialize variables with default values
    hu_mean_organ, hu_std_organ, mask_volume_ml = np.nan, np.nan, np.nan

    # Check if there are valid data points in the segmented region
    valid_data_points = hu_values_organ[segmented_data > 0]

    if valid_data_points.size > 0:
        # Calculate HU mean and standard deviation for the organ
        hu_mean_organ = valid_data_points.mean()
        hu_std_organ = valid_data_points.std()

        print("HU Mean:")
        print(hu_mean_organ)

        print("HU Standard Deviation:")
        print(hu_std_organ)

    else:
        print("No valid data points in the segmented organ region. Unable to calculate HU metrics.")

    # Calculate mask volume in milliliters
    voxel_volume_mm3 = np.prod(ct_img.header.get_zooms())  # Voxel volume in mm^3
    mask_volume_ml = np.sum(segmented_data) * voxel_volume_mm3 / 1000.0  # Convert to milliliters

    print("Mask Volume (ml):")
    print(mask_volume_ml)

    return hu_mean_organ, hu_std_organ, mask_volume_ml

def calculate_organ_suv(nifti_file_path, segmented_nifti_path, weight, radionuclide_half_life, series_time, acquisition_time, radionuclide_total_dose, rescale_slope_pet, rescale_intercept_pet):
    # Load PET NIfTI file
    pet_img = nib.load(nifti_file_path)
    pet_data = pet_img.get_fdata()

    # Apply a median filter to reduce noise
    filtered_pet_data = median_filter(pet_data, size=5)  # The size parameter may need adjustment

    # Load segmented NIfTI file
    segmented_img = nib.load(segmented_nifti_path)
    segmented_data = segmented_img.get_fdata()

    # Calculate decayed dose
    decay_time = (acquisition_time - series_time).total_seconds()
    decayed_dose = radionuclide_total_dose * (2 ** (-decay_time / radionuclide_half_life))

    # Calculate SUVbw scale factor
    suvbw_scale_factor = (weight * 1000) / decayed_dose

    # Use the filtered PET data for SUV calculations
    suvbw_organ = (filtered_pet_data * segmented_data * rescale_slope_pet + rescale_intercept_pet) * suvbw_scale_factor

    # Initialize variables with default values
    mean_suvbw_organ, suv_peak_organ, suv_max_organ = np.nan, np.nan, np.nan

    # Check if there are valid data points in the segmented region
    valid_data_points = suvbw_organ[segmented_data > 0]

    if valid_data_points.size > 0:
        # Calculate the mean SUVbw value for the organ
        mean_suvbw_organ = valid_data_points.mean()
        print("Mean SUVbw:")
        print(mean_suvbw_organ)

        # Calculate SUV Max for the organ
        suv_max_organ = valid_data_points.max()
        print("SUV Max:")
        print(suv_max_organ)

        # Find the spatial coordinates (indices) of the maximum SUV voxel
        max_suv_index = np.unravel_index(np.argmax(suvbw_organ), suvbw_organ.shape)

        # Define a small region (1 cm^3 sphere) around the maximum SUV voxel
        sphere_radius = int(np.ceil(1 / pet_img.header.get_zooms()[0]))  # in voxel units
        region_around_max_suv = suvbw_organ[
            max(0, max_suv_index[0] - sphere_radius):min(suvbw_organ.shape[0], max_suv_index[0] + sphere_radius + 1),
            max(0, max_suv_index[1] - sphere_radius):min(suvbw_organ.shape[1], max_suv_index[1] + sphere_radius + 1),
            max(0, max_suv_index[2] - sphere_radius):min(suvbw_organ.shape[2], max_suv_index[2] + sphere_radius + 1)
        ]

        # Calculate SUV Peak for the organ (average value within the 1-cm^3 sphere)
        # Calculate SUV Peak for the organ using median (more robust to noise)
        suv_peak_organ = np.median(region_around_max_suv[region_around_max_suv > 0])
        #suv_peak_organ = region_around_max_suv.mean()
        print("SUV Peak:")
        print(suv_peak_organ)

    else:
        print("No valid data points in the segmented organ region. Unable to calculate SUV metrics.")

    return mean_suvbw_organ, suv_peak_organ, suv_max_organ

5. Output storage path:

In [5]:
existing_excel_file_path = f"C:/Personal/ABX/patient_2/Results/{metadata['patient_id']}_23_01_24_Final.xlsx"

6. Main code:

In [6]:

#Get a list of all NIfTI files in the "segmented_organs" folder
segmented_nifti_paths = [os.path.join(segmented_organs_folder, file) for file in os.listdir(segmented_organs_folder) if file.endswith(".nii.gz")]

# Initialize an empty DataFrame
existing_df = pd.DataFrame(columns=[
    'Patient ID', 'Organ', 'HU Mean', 'HU Std Dev', 'SUV Mean', 'SUV Peak', 'SUV Max', 'Mask Volume (ml)'
])

# Iterate through segmented organ NIfTI files
for segmented_nifti_path in segmented_nifti_paths:
    # Calculate HU metrics
    hu_mean, hu_std, mask_volume_ml = calculate_hu_metrics(nifti_file_path_ct, segmented_nifti_path, metadata['rescale_slope_ct'], metadata['rescale_intercept_ct'])

    # Calculate SUV metrics
    #pet_weight = 65.0  # Assuming a constant weight for debuging
    #pet_radionuclide_half_life = 6586.2  # Assuming a constant half-life for debugging
    pet_mean_suvbw, pet_suv_peak, pet_suv_max = calculate_organ_suv(nifti_file_path_pet, segmented_nifti_path, metadata['weight'], metadata['radionuclide_half_life'], metadata['series_time'], metadata['acquisition_time'], metadata['radionuclide_total_dose'], metadata['rescale_slope_pet'], metadata['rescale_intercept_pet'])

    # Extract organ name from the segmented nifti file name
    organ_name = os.path.splitext(os.path.splitext(os.path.basename(segmented_nifti_path))[0])[0]

    # Extract patient ID from nifti_file_path
    match = re.search(r'/NIFTI/(.*?)/', nifti_file_path_ct)
    if match:
        patient_id = match.group(1)
    else:
        print("Unable to extract Patient ID from nifti_file_path.")

    # Determine the index of the next empty row
    next_index = len(existing_df) + 1

    # Append the new results to the existing DataFrame with the calculated index
    new_data = {
        'Patient ID': [patient_id],
        'Organ': [organ_name],
        'HU Mean': [hu_mean],
        'HU Std Dev': [hu_std],
        'SUV Mean': [pet_mean_suvbw],
        'SUV Peak': [pet_suv_peak],
        'SUV Max': [pet_suv_max],
        'Mask Volume (ml)': [mask_volume_ml],
    }

    new_df = pd.DataFrame(new_data)
    new_df.index = [next_index]

    existing_df = pd.concat([existing_df, new_df], ignore_index=False)

# Save the updated DataFrame to the Excel file without the index column
existing_df.to_excel(existing_excel_file_path, index=False)

# Print a message indicating successful export
print(f"HU and SUV Results appended to {existing_excel_file_path}")

HU Mean:
49.87330965513282
HU Standard Deviation:
28.33698371959376
Mask Volume (ml):
5.610900455474853
Mean SUVbw:
0.8190398437821959
SUV Max:
1.0761061442853597
SUV Peak:
1.0438493378232379


C:\Users\Kathir\AppData\Local\Temp\ipykernel_13796\1218589794.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  existing_df = pd.concat([existing_df, new_df], ignore_index=False)


HU Mean:
50.13368060542748
HU Standard Deviation:
25.16374751029642
Mask Volume (ml):
3.831834457397461
Mean SUVbw:
0.9171694022907495
SUV Max:
1.222258943859571
SUV Peak:
0.9782755981935909
HU Mean:
121.60717141265621
HU Standard Deviation:
32.241875316520826
Mask Volume (ml):
207.61575787353516
Mean SUVbw:
0.9518843883248221
SUV Max:
1.4942989300389617
SUV Peak:
1.3709464949477554
HU and SUV Results appended to C:/Personal/ABX/patient_2/Results/PETCT_4ef69de4e1_23_01_24_Final.xlsx
